In [ ]:
import sqlite3
from datetime import datetime

# Database setup
def initialize_database():
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            email TEXT UNIQUE NOT NULL,
            password TEXT NOT NULL
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS books (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            author TEXT NOT NULL,
            available INTEGER DEFAULT 1
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS borrow_requests (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            user_id INTEGER NOT NULL,
            book_id INTEGER NOT NULL,
            start_date TEXT NOT NULL,
            end_date TEXT NOT NULL,
            status TEXT DEFAULT 'Pending',
            FOREIGN KEY(user_id) REFERENCES users(id),
            FOREIGN KEY(book_id) REFERENCES books(id)
        )
    """)
    conn.commit()
    conn.close()

# Admin functions
def create_user(email, password):
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO users (email, password) VALUES (?, ?)", (email, password))
        conn.commit()
        print("User created successfully.")
    except sqlite3.IntegrityError:
        print("Error: Email already exists.")
    conn.close()

def view_all_requests():
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT br.id, u.email, b.title, br.start_date, br.end_date, br.status
        FROM borrow_requests br
        JOIN users u ON br.user_id = u.id
        JOIN books b ON br.book_id = b.id
    """)
    requests = cursor.fetchall()
    conn.close()
    if requests:
        for req in requests:
            print(f"Request ID: {req[0]}, User: {req[1]}, Book: {req[2]}, Dates: {req[3]} to {req[4]}, Status: {req[5]}")
    else:
        print("No requests found.")

def approve_deny_request(request_id, decision):
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM borrow_requests WHERE id = ?", (request_id,))
    request = cursor.fetchone()
    if not request:
        print("Request not found.")
        conn.close()
        return

    if decision.lower() == 'approve':
        cursor.execute("UPDATE borrow_requests SET status = 'Approved' WHERE id = ?", (request_id,))
        cursor.execute("UPDATE books SET available = 0 WHERE id = ?", (request[2],))
        print("Request approved.")
    elif decision.lower() == 'deny':
        cursor.execute("UPDATE borrow_requests SET status = 'Denied' WHERE id = ?", (request_id,))
        print("Request denied.")
    else:
        print("Invalid decision.")
    conn.commit()
    conn.close()

# User functions
def list_books():
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, title, author, available FROM books")
    books = cursor.fetchall()
    conn.close()
    for book in books:
        status = "Available" if book[3] else "Not Available"
        print(f"ID: {book[0]}, Title: {book[1]}, Author: {book[2]}, Status: {status}")

def request_book(user_id, book_id, start_date, end_date):
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()
    cursor.execute("SELECT available FROM books WHERE id = ?", (book_id,))
    book = cursor.fetchone()
    if not book:
        print("Book not found.")
    elif book[0] == 0:
        print("Book is not available.")
    else:
        cursor.execute("""
            INSERT INTO borrow_requests (user_id, book_id, start_date, end_date)
            VALUES (?, ?, ?, ?)
        """, (user_id, book_id, start_date, end_date))
        conn.commit()
        print("Request submitted.")
    conn.close()

def view_user_history(user_id):
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()
    cursor.execute("""
        SELECT b.title, br.start_date, br.end_date, br.status
        FROM borrow_requests br
        JOIN books b ON br.book_id = b.id
        WHERE br.user_id = ?
    """, (user_id,))
    history = cursor.fetchall()
    conn.close()
    if history:
        for entry in history:
            print(f"Book: {entry[0]}, Dates: {entry[1]} to {entry[2]}, Status: {entry[3]}")
    else:
        print("No history found.")

# Main menu
def main():
    initialize_database()
    while True:
        print("\nLibrary Management System")
        print("1. Admin: Create User")
        print("2. Admin: View All Requests")
        print("3. Admin: Approve/Deny Request")
        print("4. User: List Books")
        print("5. User: Request Book")
        print("6. User: View Borrow History")
        print("0. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            email = input("Enter user email: ")
            password = input("Enter user password: ")
            create_user(email, password)
        elif choice == '2':
            view_all_requests()
        elif choice == '3':
            request_id = int(input("Enter request ID: "))
            decision = input("Approve or Deny? ")
            approve_deny_request(request_id, decision)
        elif choice == '4':
            list_books()
        elif choice == '5':
            user_id = int(input("Enter your user ID: "))
            book_id = int(input("Enter book ID: "))
            start_date = input("Enter start date (YYYY-MM-DD): ")
            end_date = input("Enter end date (YYYY-MM-DD): ")
            request_book(user_id, book_id, start_date, end_date)
        elif choice == '6':
            user_id = int(input("Enter your user ID: "))
            view_user_history(user_id)
        elif choice == '0':
            print("Exiting system. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()



Library Management System
1. Admin: Create User
2. Admin: View All Requests
3. Admin: Approve/Deny Request
4. User: List Books
5. User: Request Book
6. User: View Borrow History
0. Exit
